In [1]:
import fasttext.util
import fasttext
import spacy
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
#load fasttext-> takes some while to load 7GB model
nlp = spacy.load('en_core_web_lg')
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

#load any hugging face model 
model_name='distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
config = model.config

In [ ]:
def mapLabels(str_labels, id_length, dict_name):
    # get embeddings for labels
    dictionary = {}
    for idx, label in enumerate(str_labels):
        embedding = ft.get_word_vector(label)
        dictionary[idx] = embedding
    with open('./' + dict_name + '_embeddings.pickle', 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
def mapTransformerLabels(str_labels, id_length, dict_name):
    # get embeddings for labels
    dictionary = {}
    for idx, label in enumerate(str_labels):
        input_ids = torch.tensor(tokenizer.encode(label)).unsqueeze(0) # Batch size 1
        embedding = model(input_ids)[0][0][0].detach().numpy() 
        dictionary[idx] = embedding
    with open('./transformers/' + dict_name + '_embeddings.pickle', 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
import numpy as np
labels_mnist = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']
labels_cifar = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
labels_fmnist = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag' , 'ankle boot']

str_labels = labels_cifar
id_length = len(str_labels)
mapTransformerLabels(str_labels, id_length, "cifar10")